# Collection of GFS Forecast Maps from Thredds Server via NCSS and Siphon

## Justin Richling
## 11/15/18

https://doi.org/10.6084/m9.figshare.5244637.v1

In [2]:
# Random Library Imports
import subprocess,os,glob,tempfile,re,imageio,webbrowser,io,sys,types,urllib,urllib2,\
time,cStringIO

# Importing Datetime Libraries
from datetime import datetime, timedelta

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import Proj 

# Numerical and Scientific Libraries
import numpy as np
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

# Accessing Data from External Databases via XLM Catalog
from siphon.ncss import NCSS
from siphon.catalog import TDSCatalog

# MetPy Libraries
import metpy
import metpy.calc as mpcalc
from metpy.units import masked_array, units
from metpy.plots import ctables
from metpy.plots import add_metpy_logo
from metpy.constants import g

# NetCDF Libraries
from netCDF4 import Dataset
from netCDF4 import num2date

# More Image Manipulation Options
from PIL import Image as PILImage

# Ipyhton Options
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML, display, Image

# Matplotlib Plotting Libraries
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm, Normalize
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LinearSegmentedColormap

# URL Manipulation
from urllib import urlopen

# PV widgits imports
import ipywidgets as widgets

# Warnings
import warnings
warnings.filterwarnings('ignore')

## Helper Functions

In [3]:
# Thanks to the crew over at Metpy for this handy little function
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

In [4]:
# MetPy Function
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)
    #print mxy,mxx

    for i in range(len(mxy)):
        #ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24,
        #        clip_on=True, horizontalalignment='center', verticalalignment='center',
        #        transform=transform)
        #ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
        #        '\n' + str(np.int(data[mxy[i], mxx[i]])),
        #        color=color, size=12, clip_on=True, fontweight='bold',
        #        horizontalalignment='center', verticalalignment='top', transform=transform)
        
        ax.text(lon[mxx[i]], lat[mxy[i]], symbol, color=color, size=24,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(lon[mxx[i]], lat[mxy[i]],
                '\n' + str(np.int(data[mxy[i], mxx[i]])),
                color=color, size=12, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set the Map Projection

In [ ]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Add Map Features
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon0,lon1,lat0,lat1]
extent = [-130., -70, 20., 60.]

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set a list for the 24 hour forecast time steps

In [ ]:
now = datetime.utcnow()
#now = datetime(2018,12,27,21,0)
today_day = int('{0:%d}'.format(now))
today_year = int('{0:%Y}'.format(now))
today_month = int('{0:%m}'.format(now))
print today_day,today_year,today_month

forecast_times = []
for i in range(4,8):
    forecast_times.append(datetime(today_year,today_month,today_day,i*3,0))
for i in range(0,5):
    forecast_times.append(datetime(today_year,today_month,today_day+1,i*3,0))
forecast_times

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Figure out where the saved maps will go

In [ ]:
# Set a path to save the plots with string format for the date to set the month and day 
im_save_path ="/path/to/saved files/"
print im_save_path

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Start at the top of the thredds catalog

In [5]:
from siphon.catalog import TDSCatalog
top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
for ref in top_cat.catalog_refs:
    print(ref)


Forecast Model Data
Forecast Products and Analyses
Observation Data
Radar Data
Satellite Data
Unidata case studies


## Model Forecast Data

In [6]:
ref = top_cat.catalog_refs['Forecast Model Data']
ref.href

u'http://thredds.ucar.edu/thredds/idd/forecastModels.xml'

In [7]:
new_cat = ref.follow()
list(new_cat.catalog_refs)

['GEFS Members - Analysis',
 'GEFS Members - Forecasts',
 'GEFS Derived Forecast Products',
 'GFS Quarter Degree Analysis',
 'GFS Quarter Degree Forecast',
 'GFS Half Degree Analysis',
 'GFS Half Degree Forecast',
 'GFS One Degree Analysis',
 'GFS One Degree Forecast',
 'GFS Global 1.0 Degree (NOAAPORT)',
 'GFS Pacific 40km',
 'GFS Pacific 20km',
 'GFS Puerto Rico Half Degree',
 'GFS Puerto Rico Quarter Degree',
 'GFS CONUS 80km',
 'GFS CONUS 20km',
 'GFS CONUS 95km',
 'GFS Alaska 20km',
 'GSD HRRR CONUS 3km wrfprs',
 'GSD HRRR CONUS 3km surface',
 'NCEP HRRR CONUS 2.5km Analysis',
 'NCEP HRRR CONUS 2.5km',
 'NAM Alaska 11km',
 'NAM Alaska 45km from NOAAPORT',
 'NAM Alaska 45km from CONDUIT',
 'NAM Alaska 95km',
 'NAM CONUS 12km from NOAAPORT',
 'NAM CONUS 12km from CONDUIT',
 'NAM CONUS 20km',
 'NAM CONUS 40km',
 'NAM CONUS 80km',
 'NAM Polar 90km',
 'NAM Fireweather Nested',
 'Rapid Refresh CONUS 13km',
 'Rapid Refresh CONUS 20km',
 'Rapid Refresh CONUS 40km',
 'SREF CONUS 40km Ensem

## GFS Quarter Degree Data

In [8]:
model = new_cat.catalog_refs[4]
print str(model)
gfs_cat = model.follow()
list(gfs_cat.datasets)

GFS Quarter Degree Forecast


['Full Collection (Reference / Forecast Time) Dataset',
 'Best GFS Quarter Degree Forecast Time Series',
 'Latest Collection for GFS Quarter Degree Forecast']

## Best GFS Quarter Degree Forecast Time Series

In [9]:
ds = gfs_cat.datasets[1]
print "Variable Name:",ds.name
print "Path:",ds.url_path

Variable Name: Best GFS Quarter Degree Forecast Time Series
Path: grib/NCEP/GFS/Global_0p25deg/Best


<h2>----------------------------------------------//---------------------------------------------------------</h2>
<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h1><font style="font-size:32px"><center>-- Plotting all of the GFS forecast hours for the current day --</center></font></h1>

<h2><font><center>-- MSLP, Surface Winds and Temps --</center></font></h2>

In [ ]:
for i in forecast_times:
    ds = gfs_cat.datasets[1]
    subset = ds.subset()
    query_data = subset.query()
    query_data.lonlat_box(west=-130, east=-50, south=10, north=60)

# Allow for NetCDF files
    query_data.accept('netcdf4')
    query_data.time(i)
    data = query_data.variables("Pressure_reduced_to_MSL_msl",
               'Apparent_temperature_height_above_ground',
                'u-component_of_wind_height_above_ground',
               'v-component_of_wind_height_above_ground')

# Finally attempt to access the data
    data = subset.get_data(query_data)
    #for i in data.variables.keys(): print "Variables:",i,"\n"
    
    # Pull out variables you want to use
    mslp = data.variables['Pressure_reduced_to_MSL_msl'][:].squeeze()
    temp = units.K * data.variables['Apparent_temperature_height_above_ground'][:].squeeze()
    u_wind = units('m/s') * data.variables['u-component_of_wind_height_above_ground'][:].squeeze()
    v_wind = units('m/s') * data.variables['v-component_of_wind_height_above_ground'][:].squeeze()
    lat = data.variables['lat'][:].squeeze()
    lon = data.variables['lon'][:].squeeze()
    lats = data.variables['lat'][:]
    lons = data.variables['lon'][:]
    time_var = data.variables[find_time_var(data.variables['Pressure_reduced_to_MSL_msl'])]

# Convert winds to knots
    u_wind.ito('kt')
    v_wind.ito('kt')

# Convert number of hours since the reference time into an actual date
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"

    lev_10m = np.where(data.variables['height_above_ground3'][:] == 10)[0][0]
    u_wind_10m = u_wind[lev_10m]
    v_wind_10m = v_wind[lev_10m]

# Combine 1D latitude and longitudes into a 2D grid of locations
    lon_2d, lat_2d = np.meshgrid(lon, lat)
    
# Smooth MSLP a little
# Be sure to only put in a 2D lat/lon or Y/X array for smoothing
    smooth_mslp = ndimage.gaussian_filter(mslp, sigma=3, order=0) * units.Pa
    smooth_mslp.ito('hPa')

    
    
    # Create new figure
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax = plt.subplot(111, projection=plotcrs)

#Set the lat and lon boundaries
    ax.set_extent(extent, datacrs)

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='blue', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
    plt.title('MSLP (hPa), 2m Temperature (F), Wind Barbs (kt)', fontsize=16,loc='left')
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final), fontsize=16,loc='right')

                                            # MSLP
#---------------------------------------------------------------------------------------------------
#clev_mslp = np.arange(0, 1200, 3)

#cs = ax.contour(lon_2d, lat_2d, smooth_mslp, clev_mslp, colors='k', linewidths=3,
#    linestyles='solid', transform=datacrs) # cmap='rainbow

#plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
#           rightside_up=True, use_clabeltext=True,colors='k')

# Plot Absolute Vorticity
#clev_abVort = np.arange(Max,Min,0.00001)
#abVort = ax.contourf(lon_2d, lat_2d, abs_vort[5,:,:],transform=datacrs,cmap='BrBG')
#cbaxes = fig.add_axes([0.92, 0.233, 0.03, 0.54]) # [left, bottom, width, height]
#cbar = plt.colorbar(abVort, orientation='vertical',cax=cbaxes)
#plt.colorbar(cs2, orientation='vertical', fraction=0.0311, pad=0.02)


                                            # 2m Temperatures
#---------------------------------------------------------------------------------------------------
# Plot 2m Temperature Contours
    clevtemp = np.arange(-30, 90, 2)

# Uncomment for contours instead of contour fill (below)
#cs2 = ax.contour(lon_2d, lat_2d, temp.to(units('degF')), clevtemp,
#                 cmap='gist_rainbow_r', linewidths=1.25,
#                 transform=datacrs)  #linestyles='dotted'

    cs2 = ax.contourf(lon_2d, lat_2d, temp.to(units('degF')),clevtemp,
                 transform=datacrs,cmap='gist_rainbow_r')

#cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]

#cbar = plt.colorbar(cs2, orientation='horizontal',cax=cbaxes)

#plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
#           rightside_up=True, use_clabeltext=True)

                                            # 10m Winds
#---------------------------------------------------------------------------------------------------
    ax.barbs(lon_2d, lat_2d, u_wind_10m.magnitude, v_wind_10m.magnitude,
         length=6, regrid_shape=20, pivot='middle', transform=datacrs,barbcolor='k')

    #plt.show()
    outfile=im_save_path+"SIMPLE_MSLP_Temps_Winds_"+file_time+".png"
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h2><font><center>-- Highs/Lows and 1000-500mb Thickness --</center></font></h2>

In [ ]:
for i in forecast_times:
    ds = gfs_cat.datasets[1]
    subset = ds.subset()
    query_data = subset.query()
    query_data.lonlat_box(west=-130, east=-50, south=10, north=60)

# Allow for NetCDF files
    query_data.accept('netcdf4')
    query_data.time(i)
    data = query_data.variables('Geopotential_height_isobaric',
               'Pressure_reduced_to_MSL_msl')

# Finally attempt to access the data
    data = subset.get_data(query_data)
    
# Grab pressure levels
    plev = list(data.variables['isobaric'][:])

# Grab lat/lons and make all lons 0-360
    lats = data.variables['lat'][:]
    lons = data.variables['lon'][:]
    lons[lons < 0] = 360 + lons[lons < 0]

# Grab MSLP and smooth, use MetPy Units module for conversion
    EMSL = data.variables['Pressure_reduced_to_MSL_msl'][:] * units.Pa
    EMSL.ito('hPa')
    mslp = gaussian_filter(EMSL[0], sigma=3.0)

# Grab pressure level data
    hght_1000 = data.variables['Geopotential_height_isobaric'][0, plev.index(100000.0)]
    hght_500 = data.variables['Geopotential_height_isobaric'][0, plev.index(50000.0)]
#mb500 = data.variables['Geopotential_height_isobaric'][0,::plev.index(50000.0)]*units.Pa
#mb500.ito('hPa')
    Height500mb = gaussian_filter(hght_500[0], sigma=3.0)

# Calculate and smooth 1000-500 hPa thickness
    thickness_1000_500 = gaussian_filter(hght_500 - hght_1000, sigma=3.0)
    
# Convert number of hours since the reference time into an actual date
    time_var = data.variables[find_time_var(data.variables['Pressure_reduced_to_MSL_msl'])]
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"    
    
# Create new figure
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map 
    ax = plt.subplot(111, projection=plotcrs)

# Set extent and plot map lines
    ax.set_extent(extent, datacrs)
#ax.set_extent([-75, -30.5, 0.5, 28.5])

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
    plt.title('MSLP (hPa) with Highs and Lows, 1000-500 hPa Thickness (m)', loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final), loc='right',fontsize=16)

                                            # 1000-500mb Thickness
#---------------------------------------------------------------------------------------------------
    clevs = (np.arange(0, 5400, 60), 5400, np.arange(5460, 7000, 60))
    colors = ('tab:blue', 'b', 'tab:red')
    kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}
    for clevthick, color in zip(clevs, colors):
        cs = ax.contour(lons, lats, thickness_1000_500, levels=clevthick, colors=color,
                    linewidths=1.0, linestyles='dashed', transform=datacrs)
        plt.clabel(cs, **kw_clabels)

                                            # MSLP
#---------------------------------------------------------------------------------------------------
    clevmslp = np.arange(800., 1120., 4)
    cs2 = ax.contour(lons, lats, mslp, clevmslp, colors='k', linewidths=1.25,
                 linestyles='solid', transform=datacrs)
    plt.clabel(cs2, **kw_clabels)

                                            # High and Low Symbols
#---------------------------------------------------------------------------------------------------
    plot_maxmin_points(lons, lats, mslp, 'max', 50, symbol='H', color='b',  transform=datacrs)
    plot_maxmin_points(lons, lats, mslp, 'min', 25, symbol='L', color='r', transform=datacrs)
    
#plt.show()
    outfile=im_save_path+"HL_Thickness_Model_"+file_time+".png"
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
print "done."

<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h2><font><center>-- 250mb Heights and Jet Streaks --</center></font></h2>

In [ ]:
for i in forecast_times:
    ds = gfs_cat.datasets[1]
    subset = ds.subset()
    query_data = subset.query()
    query_data.lonlat_box(west=-130, east=-50, south=10, north=60)

# Allow for NetCDF files
    query_data.accept('netcdf4')
    query_data.time(i)
    data = query_data.variables('Geopotential_height_isobaric',
               'Pressure_reduced_to_MSL_msl',
               'u-component_of_wind_isobaric',
               'v-component_of_wind_isobaric')

# Finally attempt to access the data
    data = subset.get_data(query_data)
    
    lev_250 = np.where(data.variables['isobaric'][:] == 25000)[0][0]

    hght_250 = data.variables['Geopotential_height_isobaric'][0, lev_250, :, :]
    u_250 = data.variables['u-component_of_wind_isobaric'][0, lev_250, :, :]
    v_250 = data.variables['v-component_of_wind_isobaric'][0, lev_250, :, :]
    
# Create a figure object, title it, and do the plots.
    fig = plt.figure(figsize = (17.,11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax6 = plt.subplot(1,1,1, projection=plotcrs)

# Add state boundaries to plot
    ax6.add_feature(states_provinces, edgecolor='b', linewidth=1)

# Add country borders to plot
    ax6.add_feature(country_borders, edgecolor='k', linewidth=1)

# Convert number of hours since the reference time into an actual date
    time_var = data.variables[find_time_var(data.variables['v-component_of_wind_isobaric'])]
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z" 
    
# Plot Title
    plt.title('250mb Heights and Jet Streaks (m/s)',loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)


                                        # Heights
#---------------------------------------------------------------------------------------------------

    MIN = hght_250.min()
    MAX = hght_250.max()

    #print hght_250.min(),hght_250.max()
    hght_250 = ndimage.gaussian_filter(hght_250, sigma=3, order=0) * units.meter

    clev250 = np.arange(MIN, MAX, 80)
    cs = ax6.contour(lon, lat, hght_250.m, clev250, colors='black', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
#plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
#           rightside_up=True, use_clabeltext=True)

                                        # Winds
#---------------------------------------------------------------------------------------------------
#lon_slice = slice(None, None, 7)
#lat_slice = slice(None, None, 7)
#ax4.barbs(lon[lon_slice], lat[lat_slice],
#         u_250[lon_slice, lat_slice].magnitude,
#         v_250[lon_slice, lat_slice].magnitude,
#         transform=ccrs.PlateCarree(), zorder=2)

    wspd250 = mpcalc.get_wind_speed(u_250, v_250)
#print wspd250.min()
    clevsped250 = np.arange(50, 100, 1)
    cf = ax6.contourf(lon, lat, wspd250, clevsped250, cmap="gist_ncar", transform=ccrs.PlateCarree())
#cbar = plt.colorbar(cf, cax=cax, orientation='horizontal', extend='max', extendrect=True,pad=0.2)
#cbaxes = fig.add_axes(colorbar_axis) 

#cbar = plt.colorbar(cf, orientation='horizontal',cax=cbaxes)

    ax6.set_extent(extent, datacrs)


#---------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------

    fig.savefig(im_save_path+"SIMPLE_250mb_Heights_Winds_"+file_time+".png",
            bbox_inches='tight',dpi=120)
print 'done.'

<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h2><font><center>-- 1000 and 500mb Heights --</center></font></h2>

In [ ]:
for i in forecast_times:
    ds = gfs_cat.datasets[1]
    subset = ds.subset()
    query_data = subset.query()
    query_data.lonlat_box(west=-130, east=-50, south=10, north=60)

# Allow for NetCDF files
    query_data.accept('netcdf4')
    query_data.time(i)
    data = query_data.variables('Geopotential_height_isobaric')

# Finally attempt to access the data
    data = subset.get_data(query_data)
    
    lev_500 = np.where(data.variables['isobaric'][:] == 50000)[0][0]
    hght_500 = data.variables['Geopotential_height_isobaric'][0, lev_500, :, :]
    
    # Create a figure object, title it, and do the plots.
    fig = plt.figure(figsize = (17.,11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax5 = plt.subplot(1,1,1, projection=plotcrs)
    ax5.set_extent(extent, datacrs)

# Add state boundaries to plot
    ax5.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
    ax5.add_feature(country_borders, edgecolor='black', linewidth=1)

# Convert number of hours since the reference time into an actual date
    time_var = data.variables[find_time_var(data.variables['Geopotential_height_isobaric'])]
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    
# Plot Title
    plt.title('1000 and 500mb Heights (m)',loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                        # Heights - 1000mb
#---------------------------------------------------------------------------------------------------
    lev_1000 = np.where(data.variables['isobaric'][:] == 100000)[0][0]

    hght_1000 = data.variables['Geopotential_height_isobaric'][0, lev_1000, :, :]
    MIN = hght_1000.min()
    MAX = hght_1000.max()
#print hght_1000.min(),hght_1000.max()
    hght_1000 = ndimage.gaussian_filter(hght_1000, sigma=3, order=0) * units.meter

    clev1000 = np.arange(MIN, MAX, 50)
    cs = ax5.contour(lon, lat, hght_1000.m, clev1000, colors='blue', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
    plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

                                        # Heights - 500mb
#---------------------------------------------------------------------------------------------------
    MIN = hght_500.min()
    MAX = hght_500.max()
    #print hght_500.min(),hght_500.max()
    hght_500 = ndimage.gaussian_filter(hght_500, sigma=3, order=0) * units.meter

    clev500 = np.arange(MIN, MAX, 70)
    cs = ax5.contour(lon, lat, hght_500.m, clev500, colors='black', linewidths=2.0,
                linestyles='solid', transform=ccrs.PlateCarree())
    plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)


                                        # Vorticity
#---------------------------------------------------------------------------------------------------
#uwnd_500 = data.variables['u-component_of_wind_isobaric'][0, lev_500, :, :] * units('m/s')
#vwnd_500 = data.variables['v-component_of_wind_isobaric'][0, lev_500, :, :] * units('m/s')
#dx, dy = mpcalc.lat_lon_grid_spacing(lon, lat)#

#lat = data.variables['lat'][:]
#f = mpcalc.coriolis_parameter(np.deg2rad(lat)).to(units('1/sec'))

#avor = mpcalc.vorticity(uwnd_500, vwnd_500, dx, dy, dim_order='yx') + f
#print avor.units
#MAX = avor2.max()
#MIN = avor2.min()
#print MAX*units('dimensionless'),MIN*units('dimensionless')

#avor2 = ndimage.gaussian_filter(avor, sigma=3, order=0) * units('1/s')


#vort_adv = mpcalc.advection(avor2, [uwnd_500, vwnd_500], (dx, dy), dim_order='yx') * 1e9
# Plot Colorfill of Vorticity Advection
#clev_avoradv = np.arange(MIN, MAX, 5)
#cf = ax5.contourf(lon, lat, vort_adv.m, clev_avoradv[clev_avoradv != 0], extend='both',
#                 cmap='bwr', transform=ccrs.PlateCarree())
#cax = plt.subplot(gs[1])
#cb = plt.colorbar(cf, cax=cax, orientation='horizontal', extendrect='True', ticks=clev_avoradv)
#cb.set_label(r'$1/s^2$', size='large')

#---------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------

    fig.savefig(im_save_path+"1000_500mb_Heights_"+file_time+".png",
            bbox_inches='tight',dpi=120)
print 'done.'

<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h2><font><center>-- Isentropic Analysis --</center></font></h2>

<ul>
  <li>Isentropic Heights and Winds</li>
  <li>Isentropic Heights, Winds, and Mixing Ratio</li>
  <li>Isentropic Heights, Winds, and Omega</li>
</ul>

In [ ]:
for i in forecast_times:
    cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/'
                 'NCEP/GFS/Global_0p5deg/catalog.xml')
    best = cat.datasets['Best GFS Half Degree Forecast Time Series']

    subset_access = best.subset()
    query = subset_access.query()

    query.time(i)
    #print query.time(datetime.utcnow())
    #print datetime.utcnow()
    #time = str(datetime.utcnow())[:-7]
    #print time

    query.variables('Temperature_isobaric', 'Geopotential_height_isobaric',
                'u-component_of_wind_isobaric', 'v-component_of_wind_isobaric',
                'Relative_humidity_isobaric','Absolute_vorticity_isobaric')
#230., 295., 15., 45.
#query.lonlat_box(west=230., east=295., south=15., north=45.)
    query.lonlat_box(west=-130, east=-50, south=10, north=60)
    query.accept('netcdf4')
    data = subset_access.get_data(query)

    
    lat = data.variables['lat'][:]
    lon = data.variables['lon'][:]
    press = data.variables['isobaric'][:] * units.Pa
    temperature = data.variables['Temperature_isobaric'][0] * units.kelvin
    rh = data.variables['Relative_humidity_isobaric'][0] * units.percent
    height = data.variables['Geopotential_height_isobaric'][0] * units.meter
    u = data.variables['u-component_of_wind_isobaric'][0] * units('m/s')
    v = data.variables['v-component_of_wind_isobaric'][0] * units('m/s')
    #vort =  data.variables['Absolute_vorticity_isobaric'][0] * units('1/s')  
    TEMP = 295
    isen_level = np.array([TEMP]) * units.kelvin
    isen_press, isen_u, isen_v = mpcalc.isentropic_interpolation(isen_level, press,
            temperature, u, v)
# Need to squeeze() out the size-1 dimension for the isentropic level
    isen_press = isen_press.squeeze()
    isen_u = isen_u.squeeze()
    isen_v = isen_v.squeeze()
    #isen_vort = isen_vort.squeeze()
    
# Needed to make numpy broadcasting work between 1D pressure and other 3D arrays
    pressure_for_calc = press[:, None, None]

# Calculate mixing ratio using something from mpcalc
    mixing = mpcalc.mixing_ratio_from_relative_humidity(rh, temperature, pressure_for_calc)

# Take the return and convert manually to units of 'dimenionless'
    mixing.ito('dimensionless')

# Interpolate all the data
    isen_level = np.array([295]) * units.kelvin
    ret = mpcalc.isentropic_interpolation(isen_level, press, temperature, mixing, u, v)
    isen_press, isen_mixing, isen_u, isen_v = ret

# Squeeze the returned arrays
    isen_press = isen_press.squeeze()
    isen_mixing = isen_mixing.squeeze()
    isen_u = isen_u.squeeze()
    isen_v = isen_v.squeeze()
    
# Create a plot and basic map projection
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Convert number of hours since the reference time into an actual date
    time_var = data.variables[find_time_var(data.variables['Geopotential_height_isobaric'])]
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"    
    
# Plot Title
    plt.title('{}K Isentrope Heights and Winds (kts)'.format(TEMP),loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
# Contour the pressure values for the isentropic level. We keep the handle
# for the contour so that we can have matplotlib label the contours

    levels = np.arange(300, 1000, 10)
    cntr = ax.contour(lon, lat, isen_press, transform=datacrs,
                 cmap='rainbow',levels=levels,linewidths=2) #ccrs.PlateCarree()
#ax.clabel(cntr, fmt='%.0f',colors='black')

#cbaxes = fig.add_axes(colorbar_axis)
#cbar = plt.colorbar(cntr, orientation='horizontal',cax=cbaxes)

                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
# Set up slices to subset the wind barbs--the slices below are the same as `::5`
# We put these here so that it's easy to change and keep all of the ones below matched
# up.

    lon_slice = slice(None, None, 7)
    lat_slice = slice(None, None, 7)
    ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=2,length=7) # barbcolor="" optional call

    ax.set_extent(extent, datacrs)

    #plt.show()

    GFS_Isentrope = im_save_path+"GFS/Isentropes/"
    if not os.path.isdir(GFS_Isentrope):
        os.makedirs(GFS_Isentrope)
    fig.savefig(GFS_Isentrope+"Isentropes_Winds_"+file_time+".png",
            bbox_inches='tight',dpi=120)
    
######################################################################################################## 

    # Create a plot and basic map projection
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Convert number of hours since the reference time into an actual date
    time_var = data.variables[find_time_var(data.variables['Geopotential_height_isobaric'])]
    time_final = num2date(time_var[:].squeeze(), time_var.units)
    print str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"
    file_time = str(time_final)[:4]+"_"+str(time_final)[5:7]+"_"+str(time_final)[8:10]+"_"+str(time_final)[11:13]+"Z"    
    
# Plot Title
    plt.title('{}K Isentrope Pressure, Winds (kts) and Mixing Ratio'.format(TEMP),loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
    levels = np.arange(300, 1000, 20)
    cntr = ax.contour(lon, lat, isen_press, transform=datacrs,
                  cmap='rainbow', levels=levels,linewidths=2.0)#colors='black'
    ax.clabel(cntr, fmt='%.0f')

                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
    lon_slice = slice(None, None, 8)
    lat_slice = slice(None, None, 8)
    ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=3,barbcolor='k')


                                           # Isentropic Mixing Ratio
#---------------------------------------------------------------------------------------------------
# Contourf the mixing ratio values
    mixing_levels = [0.001, 0.002, 0.004, 0.006, 0.010, 0.012, 0.014, 0.016, 0.020]
    cntr2 = ax.contourf(lon, lat, isen_mixing, transform=datacrs,
            levels=mixing_levels, cmap='YlGn')

    ax.set_extent(extent, datacrs)

#cbaxes = fig.add_axes(colorbar_axis) # [left, bottom, width, height]
#cbar = plt.colorbar(cntr2, orientation='horizontal',cax=cbaxes)

    #plt.show()


    GFS_Isentrope = im_save_path+"GFS/Isentropes_MixRatio/"
    if not os.path.isdir(GFS_Isentrope):
        os.makedirs(GFS_Isentrope)
    fig.savefig(GFS_Isentrope+"Isentropes_Winds_MixingRatio_"+file_time+".png",
            bbox_inches='tight',dpi=120)

########################################################################################################
    isen_press.units, isen_u.units, isen_v.units
    dx, dy = mpcalc.lat_lon_grid_spacing(lon, lat)
    dy = -dy

# Filter and re-attach units
    isen_press = gaussian_filter(isen_press.squeeze(), sigma=2.0) * units.hPa
    isen_u = gaussian_filter(isen_u.squeeze(), sigma=2.0) * units('m/s')
    isen_v = gaussian_filter(isen_v.squeeze(), sigma=2.0) * units('m/s')
    lift = -mpcalc.advection(isen_press, [isen_u, isen_v], [dx, dy], dim_order='yx')

# Create a plot and basic map projection
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 30, 940, size='small')

# Add the map and set the extent
    ax = plt.subplot(111, projection=plotcrs)

# Add state boundaries to plot
    ax.add_feature(states_provinces, edgecolor='k', linewidth=1)

# Add country borders to plot
    ax.add_feature(country_borders, edgecolor='black', linewidth=1)

# Plot Title
    plt.title('{}K Isentropes, Winds (kts) and Omega'.format(TEMP),loc='left',fontsize=16)
    plt.title(' {0:%d %B %Y %H:%MZ}'.format(time_final),loc='right',fontsize=16)

                                            # Isentropic Pressure
#---------------------------------------------------------------------------------------------------
    levels = np.arange(300, 1000, 50)
#cntr = ax.contour(lon, lat, isen_press, transform=ccrs.PlateCarree(), colors='black', levels=levels)
    cntr = ax.contour(lon, lat, isen_press, transform=ccrs.PlateCarree(), cmap='rainbow', levels=levels,linewidths=2.0)
    ax.clabel(cntr, fmt='%.0f')


                                            # Isentropic Winds
#---------------------------------------------------------------------------------------------------
    lon_slice = slice(None, None, 7)
    lat_slice = slice(None, None, 7)
    ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].to('knots').magnitude,
         isen_v[lon_slice, lat_slice].to('knots').magnitude,
         transform=ccrs.PlateCarree(), zorder=2)


                                            # Omega
#---------------------------------------------------------------------------------------------------
    levels = np.arange(-10, 10,2)
    cs = ax.contourf(lon, lat, lift.to('microbar/s'), levels=levels, cmap='RdBu',
                 transform=ccrs.PlateCarree())#, extend='both')
#cbaxes = fig.add_axes(colorbar_axis) 
#cbar = plt.colorbar(cs, orientation='horizontal',cax=cbaxes)

    ax.set_extent(extent, datacrs)
    
    fig.savefig(im_save_path+"Isentropes_Winds_MixingRatio_Omega_"+file_time+".png",
            bbox_inches='tight',dpi=120)

print "done."

<h2>----------------------------------------------//---------------------------------------------------------</h2>